# Milestone P4 : Creative Extension

## Libraries

In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import math
import matplotlib.image as mpimg
%matplotlib inline

## The data

In [2]:
DATA_FOLDER = 'data/'

GOWALLA_CHECKINS_DATASET = DATA_FOLDER+"G_loc_country.csv"
BRIGHTKITE_CHECKINS_DATASET = DATA_FOLDER+"B_loc_country.csv"
POPULATION_DATASET = DATA_FOLDER+"pop_per_country.csv"
BRIGHTKITE_HOME_LOCATIONS = DATA_FOLDER+"new_home_loc_B_country.csv"
GOWALLA_HOME_LOCATIONS = DATA_FOLDER+"new_home_loc_G_country.csv"
COUNTRY_CODES = DATA_FOLDER+"countrycodes.csv"

gowalla_checkins = pd.read_csv(GOWALLA_CHECKINS_DATASET, delimiter = ',', usecols = [1,2,3,4,5])
brightkite_checkins = pd.read_csv(BRIGHTKITE_CHECKINS_DATASET, delimiter = ',', usecols = [1,2,3,4,5])
population = pd.read_csv(POPULATION_DATASET, index_col = 0, names = ['country', 'population'], header = 0)
brightkite_home_locations = pd.read_csv(BRIGHTKITE_HOME_LOCATIONS, sep =',')
gowalla_home_locations = pd.read_csv(GOWALLA_HOME_LOCATIONS, sep =',')
country_codes = pd.read_csv(COUNTRY_CODES, sep =',')

## Cleaning the data

In [3]:
population.sort_values(by = ['population'], ascending = False).head(10)

,country,population
24,china,1313973713
51,india,1095351995
127,united states,298444215
52,indonesia,245452739
15,brazil,188078227
93,pakistan,165803560
9,bangladesh,147365352
101,russia,142893540
91,nigeria,131859731
59,japan,127463611


In [4]:
brightkite_home_locations.head(3)

,user,lat,lon,country
0,0,39.747913,-104.968944,US
1,1,37.599899,-122.372723,US
2,2,39.738874,-104.954143,US


In [5]:
gowalla_home_locations.head(3)

,user,lat,lon,country
0,0,30.259167,-97.750223,US
1,1,50.381005,3.430710,FR
2,2,34.052243,-118.327641,US


In [6]:
country_codes.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [7]:
gowalla_home_locations = pd.merge(gowalla_home_locations, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
gowalla_home_locations.drop(columns = ['country', 'Code'], inplace = True)
gowalla_home_locations.rename(columns = {'Name':'country'}, inplace = True)

brightkite_home_locations = pd.merge(brightkite_home_locations, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
brightkite_home_locations.drop(columns = ['country', 'Code'], inplace = True)
brightkite_home_locations.rename(columns = {'Name':'country'}, inplace = True)

In [8]:
gowalla_checkins = pd.merge(gowalla_checkins, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
gowalla_checkins.drop(columns = ['country', 'Code'], inplace = True)
gowalla_checkins.rename(columns = {'Name':'country'}, inplace = True)

brightkite_checkins = pd.merge(brightkite_checkins, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
brightkite_checkins.drop(columns = ['country', 'Code'], inplace = True)
brightkite_checkins.rename(columns = {'Name':'country'}, inplace = True)

In [9]:
brightkite_home_locations.head()

,user,lat,lon,country
0,0,39.747913,-104.968944,United States
1,1,37.599899,-122.372723,United States
2,2,39.738874,-104.954143,United States
3,3,37.780875,-122.406171,United States
4,4,60.174186,24.937477,Finland


In [10]:
gowalla_checkins['time'] = pd.to_datetime(gowalla_checkins['time'])
brightkite_checkins['time'] = pd.to_datetime(brightkite_checkins['time'])

In [11]:
brightkite_checkins.head()

,user,time,lat,lon,country
0,0,2010-10-17 01:48:53+00:00,39.747652,-104.992510,United States
1,0,2010-10-16 06:02:04+00:00,39.891383,-105.070814,United States
2,0,2010-10-16 03:48:54+00:00,39.891077,-105.068532,United States
3,0,2010-10-14 18:25:51+00:00,39.750469,-104.999073,United States
4,0,2010-10-14 00:21:47+00:00,39.752713,-104.996337,United States


## Step 1 : defining the geographic boundaries of our study

Before starting the study, we have to determine with which countries it would relevant to work. To estimate the quantity of data available in each country, we compute the number of homes, as defined in the replication, (i.e. users) that are located in each country. 

The point is to work only on countries in which the amount of data seems to be enough to have robust conclusions. 

In [12]:
brightkite_home_locations['country'] = brightkite_home_locations['country'].str.lower()
brightkite_checkins['country'] = brightkite_checkins['country'].str.lower()

gowalla_home_locations['country'] = gowalla_home_locations['country'].str.lower()
gowalla_checkins['country'] = gowalla_checkins['country'].str.lower()

In [13]:
brightkite_homes_per_country = brightkite_home_locations.groupby(['country']).count().sort_values(by = ['lat'], ascending = False)
brightkite_homes_per_country.drop(columns = ['lon', 'user'], inplace = True)
brightkite_homes_per_country.rename(columns={'lat':'count'}, inplace = True)
brightkite_homes_per_country.reset_index(inplace = True)

gowalla_homes_per_country = gowalla_home_locations.groupby(['country']).count().sort_values(by = ['lat'], ascending = False)
gowalla_homes_per_country.drop(columns = ['lon', 'user'], inplace = True)
gowalla_homes_per_country.rename(columns={'lat':'count'}, inplace = True)
gowalla_homes_per_country.reset_index(inplace = True)

gowalla_homes_per_country.head(10)

,country,count
0,united states,50639
1,sweden,18029
2,united kingdom,5435
3,germany,5213
4,norway,3662
5,canada,2210
6,saudi arabia,1917
7,thailand,1660
8,belgium,1569
9,australia,1311


In [14]:
brightkite_homes_per_country.head(10)

,country,count
0,united states,30669
1,united kingdom,3243
2,japan,2860
3,australia,1452
4,canada,1409
5,germany,1344
6,sweden,833
7,netherlands,789
8,italy,614
9,norway,593


In [15]:
brightkite_homes_per_country = brightkite_homes_per_country.merge(population, how = 'inner', on = ['country'])
gowalla_homes_per_country = gowalla_homes_per_country.merge(population, how = 'inner', on = ['country'])

In [16]:
brightkite_homes_per_country.head(3)

,country,count,population
0,united states,30669,298444215
1,united kingdom,3243,60609153
2,japan,2860,127463611


In [17]:
gowalla_homes_per_country.head(3)

,country,count,population
0,united states,50639,298444215
1,sweden,18029,9016596
2,united kingdom,5435,60609153


The number of home per country is normalized by the population of the country. 

In [18]:
brightkite_homes_per_country['count_normalized'] = brightkite_homes_per_country['count']/brightkite_homes_per_country['population']
gowalla_homes_per_country['count_normalized'] = gowalla_homes_per_country['count']/gowalla_homes_per_country['population']

In [19]:
brightkite_homes_per_country.sort_values(by = 'count_normalized', ascending = False).head(10)

,country,count,population,count_normalized
9,norway,593,4610820,0.000129
0,united states,30669,298444215,0.000103
6,sweden,833,9016596,0.000092
40,luxembourg,40,474413,0.000084
3,australia,1452,20264082,0.000072
12,finland,367,5231372,0.000070
1,united kingdom,3243,60609153,0.000054
7,netherlands,789,16491461,0.000048
4,canada,1409,33098932,0.000043
36,estonia,53,1324333,0.000040


In [20]:
gowalla_homes_per_country.sort_values(by = 'count_normalized', ascending = False).head(10)

,country,count,population,count_normalized
1,sweden,18029,9016596,0.002000
4,norway,3662,4610820,0.000794
29,luxembourg,172,474413,0.000363
0,united states,50639,298444215,0.000170
8,belgium,1569,10379067,0.000151
11,switzerland,994,7523934,0.000132
17,denmark,607,5450661,0.000111
2,united kingdom,5435,60609153,0.000090
21,singapore,392,4492150,0.000087
10,netherlands,1259,16491461,0.000076


Let's say that we will only work with countries which count_normalized value is higher to a certain rate. 

In [21]:
rate = 0.00001
brightkite_countries = brightkite_homes_per_country[brightkite_homes_per_country.count_normalized >= rate]
gowalla_countries = gowalla_homes_per_country[gowalla_homes_per_country.count_normalized >= rate]
print('With this rate,', len(brightkite_countries), 'countries are taken in the study for the Brightkite dataset.')
print('With this rate,', len(gowalla_countries), 'countries are taken in the study for the Gowalla dataset.')

With this rate, 28 countries are taken in the study for the Brightkite dataset.
With this rate, 38 countries are taken in the study for the Gowalla dataset.


In [22]:
countries = pd.merge(brightkite_countries, gowalla_countries, how = 'inner', on = ['country'], suffixes=('_brightkite', '_gowalla')) # countries that are ok both for the Gowalla and Brightkite datasets
print('With this rate,', len(countries), 'countries are taken in the study for both datasets.')

With this rate, 25 countries are taken in the study for both datasets.


In [23]:
countries.head(5)

,country,count_brightkite,population_brightkite,count_normalized_brightkite,count_gowalla,population_gowalla,count_normalized_gowalla
0,united states,30669,298444215,0.000103,50639,298444215,0.000170
1,united kingdom,3243,60609153,0.000054,5435,60609153,0.000090
2,australia,1452,20264082,0.000072,1311,20264082,0.000065
3,canada,1409,33098932,0.000043,2210,33098932,0.000067
4,germany,1344,82422299,0.000016,5213,82422299,0.000063


## Distance from home

In [24]:
gowalla_home_locations.rename(columns={'lat':'home_lat','lon':'home_lon'}, inplace = True)
brightkite_home_locations.rename(columns={'lat':'home_lat','lon':'home_lon'}, inplace = True)

In [25]:
gowalla_checkins = pd.merge(gowalla_checkins, gowalla_home_locations, how = 'left', on = ['user'], suffixes = ('_checkin', '_home'))
brightkite_checkins = pd.merge(brightkite_checkins, brightkite_home_locations, how = 'left', on = ['user'], suffixes = ('_checkin', '_home'))

In [26]:
import haversine as hvrs

In [27]:
def distance_from_home(row):
    '''computes the distance between the check-in and the user's home'''
    lat1 = row['lat']
    lat2 = row['home_lat']
    long1 = row['lon']
    long2 = row['home_lon']
    return(hvrs.haversine((lat1, long1), (lat2, long2)))

In [28]:
gowalla_checkins['distance_from_home'] = gowalla_checkins.apply(lambda row: distance_from_home(row), axis = 1)
brightkite_checkins['distance_from_home'] = brightkite_checkins.apply(lambda row: distance_from_home(row), axis = 1)

In [29]:
gowalla_checkins.head(3)

,user,time,lat,lon,country_checkin,home_lat,home_lon,country_home,distance_from_home
0,0,2010-10-19 23:55:27+00:00,30.235909,-97.795140,united states,30.259167,-97.750223,united states,5.030254
1,0,2010-10-18 22:17:43+00:00,30.269103,-97.749395,united states,30.259167,-97.750223,united states,1.107683
2,0,2010-10-17 23:42:03+00:00,30.255731,-97.763386,united states,30.259167,-97.750223,united states,1.320713


In [52]:
distance = 200 #km
gowalla_far_checkins = gowalla_checkins[gowalla_checkins.distance_from_home >= distance]
brightkite_far_checkins = brightkite_checkins[brightkite_checkins.distance_from_home >= distance]
len(gowalla_far_checkins), len(brightkite_far_checkins)

(942618, 511709)

In [53]:
gowalla_far_checkins.head(3)

,user,time,lat,lon,country_checkin,home_lat,home_lon,country_home,distance_from_home
10,0,2010-10-12 00:21:28+00:00,40.643885,-73.782806,united states,30.259167,-97.750223,united states,2447.558851
11,0,2010-10-11 20:21:20+00:00,40.741374,-73.988105,united states,30.259167,-97.750223,united states,2435.235803
12,0,2010-10-11 20:20:42+00:00,40.741388,-73.989455,united states,30.259167,-97.750223,united states,2435.130291


In [54]:
gowalla_far_checkins_per_country = gowalla_far_checkins.groupby(['country_home']).count().sort_values(by = 'user', ascending = False)
gowalla_far_checkins_per_country.rename(columns = {'user':'count'}, inplace = True)
gowalla_far_checkins_per_country.drop(gowalla_far_checkins_per_country.columns.difference(['count']), 1, inplace=True)
gowalla_far_checkins_per_country.head(10)

,count
country_home,
united states,634893
sweden,103646
germany,39033
united kingdom,29169
norway,23863
canada,15475
saudi arabia,12489
australia,7229
france,6170


In [55]:
brightkite_far_checkins_per_country = brightkite_far_checkins.groupby(['country_home']).count().sort_values(by = 'user', ascending = False)
brightkite_far_checkins_per_country.rename(columns = {'user':'count'}, inplace = True)
brightkite_far_checkins_per_country.drop(brightkite_far_checkins_per_country.columns.difference(['count']), 1, inplace=True)
brightkite_far_checkins_per_country.head(10)

,count
country_home,
united states,358789
japan,41834
united kingdom,15637
australia,10371
germany,10175
sweden,8575
canada,7780
france,6999
spain,4893


# Step 2 : countries where users travelled 

In [56]:
gowalla_far_checkins['previous_country_checkin'] = gowalla_far_checkins['country_checkin'].shift(1)
brightkite_far_checkins['previous_country_checkin'] = brightkite_far_checkins['country_checkin'].shift(1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
gowalla_far_checkins['next_country_checkin'] = gowalla_far_checkins['country_checkin'].shift(-1)
brightkite_far_checkins['next_country_checkin'] = brightkite_far_checkins['country_checkin'].shift(-1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
brightkite_far_checkins[3009:3013]

,user,time,lat,lon,country_checkin,home_lat,home_lon,country_home,distance_from_home,previous_country_checkin,next_country_checkin
18077,15,2009-04-22 20:59:28+00:00,34.082943,-118.414198,united states,40.042934,-105.285202,united states,1338.213446,united states,united states
18078,15,2009-04-22 19:58:47+00:00,33.944066,-118.408294,united states,40.042934,-105.285202,united states,1346.343495,united states,mexico
18180,15,2009-03-30 23:47:08+00:00,20.777462,-105.508102,mexico,40.042934,-105.285202,united states,2142.330261,united states,mexico
18181,15,2009-03-30 20:30:42+00:00,20.777462,-105.508102,mexico,40.042934,-105.285202,united states,2142.330261,mexico,mexico


In [59]:
def change_arrival_country(row):
    '''arrival country : return 1 if their the user moved from a country to another since the last check-in and 0 if not'''
    country = row['country_checkin']
    previous_country = row['previous_country_checkin']
    if(country != previous_country):
        return(1)
    else:
        return(0)

In [60]:
def change_departure_country(row):
    '''departure country : return 1 if their the user moves from a country to another since the last check-in and 0 if not'''
    country = row['country_checkin']
    next_country = row['next_country_checkin']
    if(country != next_country):
        return(1)
    else:
        return(0)

In [61]:
brightkite_far_checkins['change_arrival_country'] = brightkite_far_checkins.apply(lambda row: change_arrival_country(row), axis = 1)
gowalla_far_checkins['change_arrival_country'] = gowalla_far_checkins.apply(lambda row: change_arrival_country(row), axis = 1)

brightkite_far_checkins['change_departure_country'] = brightkite_far_checkins.apply(lambda row: change_departure_country(row), axis = 1)
gowalla_far_checkins['change_departure_country'] = gowalla_far_checkins.apply(lambda row: change_departure_country(row), axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [62]:
brightkite_far_checkins[3010:3014]

,user,time,lat,lon,country_checkin,home_lat,home_lon,country_home,distance_from_home,previous_country_checkin,next_country_checkin,change_arrival_country,change_departure_country
18078,15,2009-04-22 19:58:47+00:00,33.944066,-118.408294,united states,40.042934,-105.285202,united states,1346.343495,united states,mexico,0,1
18180,15,2009-03-30 23:47:08+00:00,20.777462,-105.508102,mexico,40.042934,-105.285202,united states,2142.330261,united states,mexico,1,0
18181,15,2009-03-30 20:30:42+00:00,20.777462,-105.508102,mexico,40.042934,-105.285202,united states,2142.330261,mexico,mexico,0,0
18182,15,2009-03-30 02:45:03+00:00,20.777462,-105.508102,mexico,40.042934,-105.285202,united states,2142.330261,mexico,mexico,0,0


In [81]:
brightkite_arrivals = brightkite_far_checkins[brightkite_far_checkins.change_arrival_country == 1].groupby(['country_checkin']).count().sort_values(by = 'user', ascending = False)
brightkite_arrivals.rename(columns = {'user':'count'}, inplace = True)
brightkite_arrivals.drop(brightkite_arrivals.columns.difference(['count']), 1, inplace=True)

gowalla_arrivals = gowalla_far_checkins[gowalla_far_checkins.change_arrival_country == 1].groupby(['country_checkin']).count().sort_values(by = 'user', ascending = False)
gowalla_arrivals.rename(columns = {'user':'count'}, inplace = True)
gowalla_arrivals.drop(gowalla_arrivals.columns.difference(['count']), 1, inplace=True)

In [82]:
brightkite_departures = brightkite_far_checkins[brightkite_far_checkins.change_departure_country == 1].groupby(['country_checkin']).count().sort_values(by = 'user', ascending = False)
brightkite_departures.rename(columns = {'user':'count'}, inplace = True)
brightkite_departures.drop(brightkite_departures.columns.difference(['count']), 1, inplace=True)

gowalla_departures = gowalla_far_checkins[gowalla_far_checkins.change_departure_country == 1].groupby(['country_checkin']).count().sort_values(by = 'user', ascending = False)
gowalla_departures.rename(columns = {'user':'count'}, inplace = True)
gowalla_departures.drop(gowalla_departures.columns.difference(['count']), 1, inplace=True)

In [95]:
brightkite_arrivals.head(20)

,count
country_checkin,
united states,8118
united kingdom,2240
germany,1580
japan,1432
france,1338
canada,1300
italy,911
spain,817
australia,817


In [94]:
brightkite_departures.head(20)

,count
country_checkin,
united states,8118
united kingdom,2240
germany,1580
japan,1432
france,1338
canada,1300
italy,911
spain,817
australia,817
